## LAB 09 | REDES
Martín España y Laura Tamath

In [1]:
import json
import pandas as pd

# Parte 1 – Filtrado y preprocesamiento 
Cargue  la  información  del  archivo  large_eve.json  en  una  lista,  muestre  la  cantidad  de registros total (deben ser 746, 909). Este es nuestro tráfico inicial! 

In [2]:
# data = [json.loads(line) for line in open('large_eve.json', 'r', encoding='UTF-8')]
data = pd.read_json('large_eve.json',  encoding='UTF-8', lines= True)

In [3]:
len(data)

746909

In [4]:
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos interesados  en  los  registros  DNS.  Cargue  únicamente  aquellos  registros  en  cuya  llave  se encuentra “DNS”

In [5]:
data = data.drop(data[data.event_type != 'dns'].index)

Muestre  la  nueva  cantidad  de  registros  filtrados.  Deben  ser  15749.  Esta  es  una  cantidad mucho más manejable, pero aún se debe seguir depurando la información a buscar

In [6]:
len(data)

15749

Muestre la información de 2 registros cualesquiera.

In [7]:
data.sample(2)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
257937,2017-07-22T19:03:32.667379-0500,2.127578e+14,2333133.0,dns,140.0,2001:0dbb:0c18:0014:34fa:4d67:66fe:82e9,58026.0,0000:0000:0000:0000:0000:ffff:c0a8:16a8,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365446,2017-07-22T19:22:06.956083-0500,1.628171e+15,2939243.0,dns,180.0,192.168.207.4,53.0,192.168.198.62,54310.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Debido a que la data consiste en json anidados, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe. 

In [8]:
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716752,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716753,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716893,2017-07-22T19:47:38.146503-0500,2.199316e+15,4452917.0,dns,130.0,192.168.203.67,50975.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716894,2017-07-22T19:47:38.146953-0500,2.199316e+15,4452918.0,dns,130.0,192.168.207.4,53.0,192.168.203.67,50975.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como  estamos  buscando  dominios  DGA,  debemos  filtrar  los  registros  DNS  para  aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió bajar la cantidad a 2849 registros. 

In [9]:
from pandas.io.json import json_normalize

registros_dns = [json.loads(record) for record in open('large_eve.json').readlines() if 'dns' in record]
df = json_normalize(registros_dns)
dga = df[df['dns.rrtype']==('A')][['dns.rrname','dns.rrtype']]
len(dga)

C:\Users\laury\AppData\Local\Temp\ipykernel_16488\1166767984.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(registros_dns)


2849

Filtre los dominios únicos y asígnelos a un dataframe. Debe obtener 177 registros únicos.

In [10]:
unique_records_domain = dga['dns.rrname'].unique()
len(unique_records_domain)

177

Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare como una columna nueva llamada domain_tld, y elimine todas las demás columnas. 


In [11]:
from flare.data_science.features import domain_tld_extract

tld_list = []

for item in dga['dns.rrname']:
    new_tld = domain_tld_extract(item)
    tld_list.append(new_tld)

dga = dga.assign(domain_tld=tld_list)

dga

,dns.rrname,dns.rrtype,domain_tld
0,api.wunderground.com,A,wunderground.com
1,stork79.dropbox.com,A,dropbox.com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,aoltw.net
5,api.wunderground.com,A,wunderground.com
6,safebrowsing.clients.google.com.home,A,safebrowsing.clients.google.com.home
...,...,...,...
21272,secure.informaction.com.hsd1.pa.comcast.net,A,comcast.net
21276,192.168.22.201:.stayonline.net,A,192.168.22.201:.stayonline.net
21285,wpad.aol.aoltw.net,A,aoltw.net
21297,client-software.real.com,A,real.com


 Busque  y  elimine un  dominio  que  es  un  carácter  vacío  ‘’.  También  elimine  los  siguientes dominios

- 192.168.22.110phpmyadmin
- 192.168.22.110phpmyadmin.localdomain 
- "192.168.206.56" 
- 192.168.26-27.0 
- 192.168.21.1201
- 1922.168.22.254 
- 1922.168.22.254.home 
- 192.168.21-28.0.home 
- 192.168.22.201:
- 192.168.22.201:.stayonline.net  

In [12]:
itemsToDrop = ['', '192.168.22.110phpmyadmin', '192.168.22.110phpmyadmin.localdomain',
                '"192.168.206.56"', '192.168.26-27.0', '192.168.21.1201', '1922.168.22.254',
                '1922.168.22.254.home', '192.168.21-28.0.home', '192.168.22.201:',
                '192.168.22.201:.stayonline.net']

for j in itemsToDrop:
    dga = dga.drop(dga[dga['domain_tld'] == j].index)

dga

,dns.rrname,dns.rrtype,domain_tld
0,api.wunderground.com,A,wunderground.com
1,stork79.dropbox.com,A,dropbox.com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,aoltw.net
5,api.wunderground.com,A,wunderground.com
6,safebrowsing.clients.google.com.home,A,safebrowsing.clients.google.com.home
...,...,...,...
21263,whitecell.localdomain,A,whitecell.localdomain
21272,secure.informaction.com.hsd1.pa.comcast.net,A,comcast.net
21285,wpad.aol.aoltw.net,A,aoltw.net
21297,client-software.real.com,A,real.com


Obtenga los TLD únicos, debe obtener 102 registros 

In [13]:
unique_records_domain = dga['dns.rrname'].unique()
len(unique_records_domain)

166

# Parte 2 – Data Science 

Utilice  el  clasificador  proporcionado,  debe  pasarle  como  parámetro  el  dataframe  con  la columna domain_tld, y asignar el resultado a un nuevo data. 

In [15]:
from clasificador import *
newData = clasificacion(dga)
newData

C:\Users\laury\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\laury\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- dns.rrname
- dns.rrtype

  warnings.warn(message, FutureWarning)


ValueError: could not convert string to float: 'api.wunderground.com'

Filtre aquellos considerados como DGA (valor 1 ) y muéstrelos. Recuerde que los modelos de ML ofrecen una predicción, pero los resultados pueden ser falsos positivos y falsos negativos, por  lo  que  no  podemos  fiarnos  por  completo  de  esta  clasificación  y  debemos  seguir indagando. Ahora tenemos solo 35 dominios sospechosos! 

# Parte 3 – Dominio experto 

Utilizando la lista de Cisco Umbrella incluido en Flare, indique si los dominios encontrados por el  clasificador  DGA de  la  parte  2  se  encuentran  en  este  top  1000000. Deberíamos  haber reducido los registros a 12! 

Si son considerados por el clasificador como DGA, y no están en el top de Cisco Umbrella es muy probable que sean maliciosos. Para confirmar esto podemos buscar la fecha de creación del dominio. Cree una función qué en base al dominio, devuelva la fecha de creación de este (utilice la librería whois para esto). 

Muestre la fecha de creación para cada uno de los dominios clasificados como DGA. ¿Cuáles son los dominios que podemos confirmar como sospechosos? (Deben ser cinco dominios) 

Los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y que pueden confirmarse como dominios DGA (deben ser dos). 